In [1]:
import glob
import os
import pandas as pd
import pandas_profiling as pdp

In [2]:
# 指定したファイルパスのcsvファイルを全部読み込む関数
def read_raw_all_csv(target_path = '../input/'):
    for file in glob.glob(target_path+'*.csv'):
        raw_name = 'raw_' + file.replace(target_path,'').replace('.csv','')
        exec('{} = pd.read_csv(\'{}\')'.format(raw_name, file), globals())
        print('read {} as {}'.format(file, raw_name))

In [3]:
read_raw_all_csv()

read ../input/new_merchant_transactions.csv as raw_new_merchant_transactions
read ../input/test.csv as raw_test
read ../input/merchants.csv as raw_merchants
read ../input/historical_transactions.csv as raw_historical_transactions
read ../input/train.csv as raw_train
read ../input/sample_submission.csv as raw_sample_submission


In [4]:
raw_train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749


In [5]:
pdp.ProfileReport(raw_train)

Number of variables,6
Number of observations,201917
Total Missing (%),0.0%
Total size in memory,9.2 MiB
Average record size in memory,48.0 B
Numeric,3
Categorical,1
Boolean,1
Date,0
Text (Unique),1
Rejected,0


### memo
 * targetとfeatureが全然相関してないから、このままだと全く当たらなそう

In [6]:
raw_test.head()

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1


In [7]:
pdp.ProfileReport(raw_test)

Number of variables,5
Number of observations,123623
Total Missing (%),0.0%
Total size in memory,4.7 MiB
Average record size in memory,40.0 B
Numeric,2
Categorical,1
Boolean,1
Date,0
Text (Unique),1
Rejected,0


In [23]:
# 同じcard_idが複数名いるか確認
print('number of row in raw_train is {}. number of unique card_id is {}'.format(raw_train.shape[0], len(raw_train['card_id'].unique())))
print('number of row in raw_test is {}. number of unique card_id is {}'.format(raw_test.shape[0], len(raw_test['card_id'].unique())))

number of row in raw_train is 201917. number of unique card_id is 201917
number of row in raw_test is 123623. number of unique card_id is 123623


### memo
 * trainとtestの比率は5:3くらい
 * train、testともにcard_idに重複はない

In [33]:
# trainとtestにcard_idの重複があるか確認
print(sum(raw_train['card_id'].isin(raw_test['card_id'])))
print(sum(raw_test['card_id'].isin(raw_train['card_id'])))

0
0


### memo
 * trainとtestで重複しているidは存在しない

In [36]:
raw_new_merchant_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [39]:
# month_lag（基準日との差をちょっと調べる
raw_new_merchant_transactions[raw_new_merchant_transactions['month_lag'] == 1][['month_lag', 'purchase_date']].head()

,month_lag,purchase_date
0,1,2018-03-11 14:57:36
1,1,2018-03-19 18:53:37
3,1,2018-03-07 09:43:21
4,1,2018-03-22 21:07:53
6,1,2018-03-28 19:50:19


In [40]:
raw_new_merchant_transactions[raw_new_merchant_transactions['month_lag'] == 2][['month_lag', 'purchase_date']].head()

,month_lag,purchase_date
2,2,2018-04-26 14:08:44
5,2,2018-04-02 12:53:28
7,2,2018-04-05 08:06:52
8,2,2018-04-07 18:37:40
13,2,2018-04-14 09:27:45


In [41]:
raw_new_merchant_transactions[raw_new_merchant_transactions['month_lag'] == 3][['month_lag', 'purchase_date']].head()

,month_lag,purchase_date


In [42]:
raw_new_merchant_transactions['month_lag'].unique()

array([1, 2])

### memo
 * month_lagの値は1か2のいずれか
 * lag1が3月、lag2が4月を指してるっぽい

In [8]:
pdp.ProfileReport(raw_new_merchant_transactions)

Number of variables,14
Number of observations,1963031
Total Missing (%),0.7%
Total size in memory,209.7 MiB
Average record size in memory,112.0 B
Numeric,7
Categorical,5
Boolean,1
Date,0
Text (Unique),0
Rejected,1


In [14]:
# authorized_flag = 'N'のレコードを確認する
raw_new_merchant_transactions[raw_new_merchant_transactions['authorized_flag'] == 'N'].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id


### memo
 * authorized = 'N'は存在しない

In [18]:
# merchant_id is nullのレコードを確認する
raw_new_merchant_transactions[raw_new_merchant_transactions['merchant_id'].isnull()].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
51,Y,C_ID_8c9ba46f19,17,N,0,A,2,NaN,2,1.597228,2017-12-19 08:07:34,4.0,22,20
99,Y,C_ID_0a6ceeffcc,143,N,0,A,884,NaN,1,-0.596673,2017-11-24 00:00:00,5.0,5,27
154,Y,C_ID_d15cd87e37,73,N,0,A,108,NaN,1,-0.296112,2018-03-28 14:22:31,3.0,19,34
265,Y,C_ID_faacef6d77,69,N,0,A,307,NaN,1,-0.671775,2018-02-01 09:42:32,1.0,9,19
304,Y,C_ID_e4d60467c2,48,N,1,B,703,NaN,1,-0.680791,2018-03-25 10:37:28,1.0,9,29


### memo
 * 特に変なレコードが含まれているわけではないので一旦無視

In [21]:
# installmentsの数値ごとのレコード数を調査
raw_new_merchant_transactions.groupby('installments').size()

installments
-1       55922
 0      922244
 1      836178
 2       54729
 3       44750
 4       14815
 5        9296
 6       10389
 7         863
 8        1555
 9         478
 10       8899
 11         61
 12       2850
 999         2
dtype: int64

### memo
 * -1は返品、999は例外処理的ななにかかな？
 * 偶数が多いのはセット商品的なやつかな？

In [25]:
# installments = -1を確認
raw_new_merchant_transactions[raw_new_merchant_transactions['installments'] == -1].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
10,Y,C_ID_ef55cf8d4b,69,N,-1,NaN,45,M_ID_3ffd43b4cd,1,4.452265,2018-03-31 09:55:40,1.0,9,18
189,Y,C_ID_fd55871fd8,187,N,-1,NaN,367,M_ID_2a7d376053,1,-0.329366,2018-02-10 09:33:55,1.0,15,16
270,Y,C_ID_6bf003900f,69,N,-1,NaN,884,M_ID_1b3e5a9b05,1,-0.596673,2018-03-02 18:00:10,1.0,9,27
306,Y,C_ID_e4d60467c2,69,N,-1,NaN,274,M_ID_2da56d089d,1,-0.596643,2018-03-22 13:49:08,1.0,9,36
402,Y,C_ID_cb2269c52d,213,N,-1,NaN,34,M_ID_6972fc7624,2,-0.416325,2018-04-26 14:36:11,1.0,9,38


In [33]:
# 先頭行と同じcard_idとmerchant_idを抜き出してみる
raw_new_merchant_transactions[(raw_new_merchant_transactions['card_id'] == 'C_ID_ef55cf8d4b') & (raw_new_merchant_transactions['merchant_id'] == 'M_ID_3ffd43b4cd')].head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
10,Y,C_ID_ef55cf8d4b,69,N,-1,NaN,45,M_ID_3ffd43b4cd,1,4.452265,2018-03-31 09:55:40,1.0,9,18


In [34]:
# installments = 999を確認
raw_new_merchant_transactions[raw_new_merchant_transactions['installments'] == 999]

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
231851,Y,C_ID_8b3e96516a,174,N,999,C,840,M_ID_fcb061b2bf,1,-0.027138,2017-04-24 20:04:23,1.0,9,20
631387,Y,C_ID_726ae3c2bc,174,N,999,C,840,M_ID_fcb061b2bf,2,1.032186,2017-12-23 23:57:41,1.0,9,20


### memo
* installments=-1の意味はよくわからない。ひつまず入れておく
* installments =999もよくわからない。ただ、installmentsは連続値として扱いたいし、異常値なので、外しておくのが良さそう

In [37]:
# purchase_amountの最大値や大きい値を取っているcard_idがtrainに入っているか調査
raw_new_merchant_transactions.sort_values('purchase_amount', ascending = False).head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
1676431,Y,C_ID_ca850ecf25,69,N,9,C,210,M_ID_8c654cb52e,2,263.157498,2018-04-06 15:52:38,1.0,9,35
228412,Y,C_ID_a289e7edd4,-1,Y,10,C,210,M_ID_d3efd95cc7,2,139.053747,2018-04-19 11:51:18,NaN,-1,35
893769,Y,C_ID_765f1b0537,69,N,-1,NaN,637,M_ID_f7227b1aa8,1,97.009567,2018-03-20 10:35:53,1.0,9,37
1815869,Y,C_ID_cc5b7d5b8a,69,N,5,C,210,M_ID_63154de5bf,1,76.682904,2018-01-05 17:35:21,1.0,9,35
866037,Y,C_ID_06dc178b0c,140,N,-1,NaN,222,M_ID_0f387af072,2,74.385651,2018-04-26 08:49:17,1.0,9,21


In [ ]:
# subsector_id=-1のcard_idがtrainに入っているか調査
